In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

# 从sklearn导入数据分割、交叉验证和网格搜索工具
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# 从sklearn导入线性回归相关模型
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
# 从sklearn导入数据标准化和多项式特征生成工具
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
# 从sklearn导入评估指标
from sklearn.metrics import mean_absolute_error, mean_squared_error
# 从scipy导入统计工具
from scipy import stats
# 从statsmodels导入方差膨胀因子计算工具
from statsmodels.stats.outliers_influence import variance_inflation_factor
# 导入matplotlib绘图库
import matplotlib.pyplot as plt

# 设置matplotlib支持中文显示，使用黑体
plt.rcParams['font.sans-serif'] = ['SimHei']
# 设置matplotlib正确显示负号
plt.rcParams['axes.unicode_minus'] = False

# 第一部分：数据加载

# 当前步骤
print("\n第一步：加载数据")

# 读取训练集房价数据（CSV格式）
train_price = pd.read_csv('C:/Users/25339/Desktop/pyhon/期中建模作业/ruc_Class25Q2_train_price.csv')
# 读取测试集房价数据（CSV格式）
test_price = pd.read_csv('C:/Users/25339/Desktop/pyhon/期中建模作业/ruc_Class25Q2_test_price.csv')
# 读取训练集租金数据（CSV格式）
train_rent = pd.read_csv('C:/Users/25339/Desktop/pyhon/期中建模作业/ruc_Class25Q2_train_rent.csv')
# 读取测试集租金数据（CSV格式）
test_rent = pd.read_csv('C:/Users/25339/Desktop/pyhon/期中建模作业/ruc_Class25Q2_test_rent.csv')

# 训练集房价数据的行列数
print(f"训练数据(价格): {train_price.shape}")
# 测试集房价数据的行列数
print(f"测试数据(价格): {test_price.shape}")
# 训练集租金数据的行列数
print(f"训练数据(租金): {train_rent.shape}")
# 测试集租金数据的行列数
print(f"测试数据(租金): {test_rent.shape}")

# 第二部分：处理房价数据 - 数据清洗
# 处理房价数据的标题
print("处理房价数据")

# 数据清洗步骤
print("\n数据清洗")
# 创建训练集的副本，避免修改原始数据
train_price_clean = train_price.copy()

#  识别并删除数据泄露特征
# 初始化一个空列表，用于存储可能泄露的特征名称
leakage_features = []
# 定义可能导致数据泄露的关键词列表
keywords = ['comm', 'price', '价格', '反馈', '客户']

# 遍历训练数据的所有列名
for col in train_price_clean.columns:
    # 如果列名是目标变量'Price'，跳过
    if col == 'Price':
        continue
    # 将列名转换为小写，方便匹配
    col_lower = col.lower()
    # 遍历所有关键词
    for keyword in keywords:
        # 如果列名包含关键词，说明可能存在数据泄露
        if keyword in col_lower:
            # 将该列名加入泄露特征列表
            leakage_features.append(col)
            # 跳出内层循环，避免重复添加
            break
# 识别到的数据泄露特征
print(f"  识别到可能的数据泄露特征: {leakage_features}")
# 删除识别出的数据泄露特征列，errors='ignore'表示如果列不存在也不报错
train_price_clean = train_price_clean.drop(columns=leakage_features, errors='ignore')

#  处理缺失值
# 选择所有数值型列（整数和浮点数）
numeric_cols = train_price_clean.select_dtypes(include=[np.number]).columns
# 遍历所有数值型列
for col in numeric_cols:
    # 如果该列存在缺失值
    if train_price_clean[col].isnull().sum() > 0:
        # 使用该列的中位数填充缺失值，inplace=True表示直接修改原数据
        train_price_clean[col].fillna(train_price_clean[col].median(), inplace=True)

# 选择所有类别型列（对象类型，即字符串）
categorical_cols = train_price_clean.select_dtypes(include=['object']).columns
# 遍历所有类别型列
for col in categorical_cols:
    # 如果该列存在缺失值
    if train_price_clean[col].isnull().sum() > 0:
        # 使用该列的众数（出现最多的值）填充缺失值
        train_price_clean[col].fillna(train_price_clean[col].mode()[0], inplace=True)

# 清洗后的数据维度
print(f"清洗后训练数据: {train_price_clean.shape}")

#  对测试集应用相同的清洗步骤
# 创建测试集的副本
test_price_clean = test_price.copy()
# 删除测试集中的数据泄露特征（使用训练集识别出的特征列表）
test_price_clean = test_price_clean.drop(columns=leakage_features, errors='ignore')

# 处理测试集的数值型列缺失值
numeric_cols_test = test_price_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols_test:
    if test_price_clean[col].isnull().sum() > 0:
        # 使用测试集自己的中位数填充
        test_price_clean[col].fillna(test_price_clean[col].median(), inplace=True)

# 处理测试集的类别型列缺失值
categorical_cols_test = test_price_clean.select_dtypes(include=['object']).columns
for col in categorical_cols_test:
    if test_price_clean[col].isnull().sum() > 0:
        # 使用测试集自己的众数填充
        test_price_clean[col].fillna(test_price_clean[col].mode()[0], inplace=True)

# 第三部分：特征工程 - 提取数值特征

# 特征工程步骤
print("\n特征工程")
# 创建训练集的副本用于特征工程
train_price_feat = train_price_clean.copy()

#  提取房屋户型中的数值信息
if '房屋户型' in train_price_feat.columns:
    # 使用正则表达式提取"X室"中的数字X，并转换为浮点数
    train_price_feat['室数'] = train_price_feat['房屋户型'].str.extract(r'(\d+)室').astype(float)
    # 使用正则表达式提取"X厅"中的数字X，并转换为浮点数
    train_price_feat['厅数'] = train_price_feat['房屋户型'].str.extract(r'(\d+)厅').astype(float)
    # 使用正则表达式提取"X卫"中的数字X，并转换为浮点数
    train_price_feat['卫数'] = train_price_feat['房屋户型'].str.extract(r'(\d+)卫').astype(float)
    # 将提取失败的NaN值填充为0
    train_price_feat['室数'].fillna(0, inplace=True)
    train_price_feat['厅数'].fillna(0, inplace=True)
    train_price_feat['卫数'].fillna(0, inplace=True)

#  提取楼层信息
if '所在楼层' in train_price_feat.columns:
    # 判断是否为高层（楼层描述中包含"高"字），转换为0或1
    train_price_feat['是否高层'] = train_price_feat['所在楼层'].str.contains('高', na=False).astype(int)
    # 判断是否为低层（楼层描述中包含"低"字），转换为0或1
    train_price_feat['是否低层'] = train_price_feat['所在楼层'].str.contains('低', na=False).astype(int)
    # 判断是否为中层（楼层描述中包含"中"字），转换为0或1
    train_price_feat['是否中层'] = train_price_feat['所在楼层'].str.contains('中', na=False).astype(int)

#  提取梯户比例信息
if '梯户比例' in train_price_feat.columns:
    # 使用正则表达式提取"X梯"中的数字X
    train_price_feat['梯数'] = train_price_feat['梯户比例'].str.extract(r'(\d+)梯').astype(float)
    # 使用正则表达式提取"X户"中的数字X
    train_price_feat['户数'] = train_price_feat['梯户比例'].str.extract(r'(\d+)户').astype(float)
    # 使用中位数填充提取失败的NaN值
    train_price_feat['梯数'].fillna(train_price_feat['梯数'].median(), inplace=True)
    train_price_feat['户数'].fillna(train_price_feat['户数'].median(), inplace=True)
    # 计算梯户比：梯数/(户数+1)，加1避免除零错误
    train_price_feat['梯户比'] = train_price_feat['梯数'] / (train_price_feat['户数'] + 1)

#  创建面积相关特征
if '建筑面积' in train_price_feat.columns and '套内面积' in train_price_feat.columns:
    # 确保建筑面积是数值类型，转换失败的设为NaN
    train_price_feat['建筑面积'] = pd.to_numeric(train_price_feat['建筑面积'], errors='coerce')
    # 确保套内面积是数值类型
    train_price_feat['套内面积'] = pd.to_numeric(train_price_feat['套内面积'], errors='coerce')
    # 计算公摊面积 = 建筑面积 - 套内面积
    train_price_feat['公摊面积'] = train_price_feat['建筑面积'] - train_price_feat['套内面积']
    # 计算公摊率 = 公摊面积 / 建筑面积，加1避免除零错误
    train_price_feat['公摊率'] = train_price_feat['公摊面积'] / (train_price_feat['建筑面积'] + 1)

#  创建时间特征
if '年份' in train_price_feat.columns:
    # 确保年份是数值类型
    train_price_feat['年份'] = pd.to_numeric(train_price_feat['年份'], errors='coerce')
    # 计算房龄 = 当前年份(2025) - 建造年份
    train_price_feat['房龄'] = 2025 - train_price_feat['年份']

if '建筑年代' in train_price_feat.columns:
    # 提取建筑年代中的数字（年份）
    train_price_feat['建筑年份'] = pd.to_numeric(train_price_feat['建筑年代'], errors='coerce')
    # 使用中位数填充缺失的建筑年份
    train_price_feat['建筑年份'].fillna(train_price_feat['建筑年份'].median(), inplace=True)
    # 计算建筑物房龄
    train_price_feat['建筑物房龄'] = 2025 - train_price_feat['建筑年份']

#  创建交互特征
if '建筑面积' in train_price_feat.columns and '室数' in train_price_feat.columns:
    # 计算平均每个房间的面积，加1避免除零错误
    train_price_feat['平均房间面积'] = train_price_feat['建筑面积'] / (train_price_feat['室数'] + 1)

#  创建位置特征
if 'lon' in train_price_feat.columns and 'lat' in train_price_feat.columns:
    # 确保经度是数值类型
    train_price_feat['lon'] = pd.to_numeric(train_price_feat['lon'], errors='coerce')
    # 确保纬度是数值类型
    train_price_feat['lat'] = pd.to_numeric(train_price_feat['lat'], errors='coerce')
    # 计算市中心的经度（使用中位数作为市中心）
    center_lon = train_price_feat['lon'].median()
    # 计算市中心的纬度（使用中位数作为市中心）
    center_lat = train_price_feat['lat'].median()
    # 计算到市中心的欧氏距离：sqrt((lon-center_lon)^2 + (lat-center_lat)^2)
    train_price_feat['距市中心距离'] = np.sqrt(
        (train_price_feat['lon'] - center_lon) ** 2 + (train_price_feat['lat'] - center_lat) ** 2
    )

#  对数转换（处理偏态分布）
# 定义需要进行对数转换的数值列
numeric_cols_for_log = ['建筑面积', '套内面积', '房屋总数', '楼栋总数']
# 遍历这些列
for col in numeric_cols_for_log:
    # 如果该列存在于数据中
    if col in train_price_feat.columns:
        # 确保该列是数值类型
        train_price_feat[col] = pd.to_numeric(train_price_feat[col], errors='coerce')
        # 创建新列：对原值进行log1p变换（log(1+x)），避免log(0)的问题
        train_price_feat[f'{col}_log'] = np.log1p(train_price_feat[col])

#  所有数值列类型正确
# 获取所有对象类型（字符串类型）的列
for col in train_price_feat.select_dtypes(include=['object']).columns:
    # 如果是目标变量，跳过
    if col == 'Price':
        continue
    # 将对象类型转换为数值类型
    try:
        # errors='ignore'表示转换失败时保持原值
        train_price_feat[col] = pd.to_numeric(train_price_feat[col], errors='ignore')
    except:
        # 如果转换出错，继续处理下一列
        pass

# 特征工程后的数据维度
print(f"特征工程后训练数据: {train_price_feat.shape}")

#  对测试集应用相同的特征工程
# 创建测试集的副本
test_price_feat = test_price_clean.copy()

# 提取房屋户型信息
if '房屋户型' in test_price_feat.columns:
    test_price_feat['室数'] = test_price_feat['房屋户型'].str.extract(r'(\d+)室').astype(float)
    test_price_feat['厅数'] = test_price_feat['房屋户型'].str.extract(r'(\d+)厅').astype(float)
    test_price_feat['卫数'] = test_price_feat['房屋户型'].str.extract(r'(\d+)卫').astype(float)
    test_price_feat['室数'].fillna(0, inplace=True)
    test_price_feat['厅数'].fillna(0, inplace=True)
    test_price_feat['卫数'].fillna(0, inplace=True)

# 提取楼层信息
if '所在楼层' in test_price_feat.columns:
    test_price_feat['是否高层'] = test_price_feat['所在楼层'].str.contains('高', na=False).astype(int)
    test_price_feat['是否低层'] = test_price_feat['所在楼层'].str.contains('低', na=False).astype(int)
    test_price_feat['是否中层'] = test_price_feat['所在楼层'].str.contains('中', na=False).astype(int)

# 提取梯户比例
if '梯户比例' in test_price_feat.columns:
    test_price_feat['梯数'] = test_price_feat['梯户比例'].str.extract(r'(\d+)梯').astype(float)
    test_price_feat['户数'] = test_price_feat['梯户比例'].str.extract(r'(\d+)户').astype(float)
    test_price_feat['梯数'].fillna(test_price_feat['梯数'].median(), inplace=True)
    test_price_feat['户数'].fillna(test_price_feat['户数'].median(), inplace=True)
    test_price_feat['梯户比'] = test_price_feat['梯数'] / (test_price_feat['户数'] + 1)

# 创建面积特征
if '建筑面积' in test_price_feat.columns and '套内面积' in test_price_feat.columns:
    test_price_feat['建筑面积'] = pd.to_numeric(test_price_feat['建筑面积'], errors='coerce')
    test_price_feat['套内面积'] = pd.to_numeric(test_price_feat['套内面积'], errors='coerce')
    test_price_feat['公摊面积'] = test_price_feat['建筑面积'] - test_price_feat['套内面积']
    test_price_feat['公摊率'] = test_price_feat['公摊面积'] / (test_price_feat['建筑面积'] + 1)

# 创建时间特征
if '年份' in test_price_feat.columns:
    test_price_feat['年份'] = pd.to_numeric(test_price_feat['年份'], errors='coerce')
    test_price_feat['房龄'] = 2025 - test_price_feat['年份']

if '建筑年代' in test_price_feat.columns:
    test_price_feat['建筑年份'] = pd.to_numeric(test_price_feat['建筑年代'], errors='coerce')
    test_price_feat['建筑年份'].fillna(test_price_feat['建筑年份'].median(), inplace=True)
    test_price_feat['建筑物房龄'] = 2025 - test_price_feat['建筑年份']

# 创建交互特征
if '建筑面积' in test_price_feat.columns and '室数' in test_price_feat.columns:
    test_price_feat['平均房间面积'] = test_price_feat['建筑面积'] / (test_price_feat['室数'] + 1)

# 创建位置特征（使用训练集的中心点）
if 'lon' in test_price_feat.columns and 'lat' in test_price_feat.columns:
    test_price_feat['lon'] = pd.to_numeric(test_price_feat['lon'], errors='coerce')
    test_price_feat['lat'] = pd.to_numeric(test_price_feat['lat'], errors='coerce')
    # 注意：使用训练集计算的中心点，确保一致性
    test_price_feat['距市中心距离'] = np.sqrt(
        (test_price_feat['lon'] - center_lon) ** 2 + (test_price_feat['lat'] - center_lat) ** 2
    )

# 对数转换
for col in numeric_cols_for_log:
    if col in test_price_feat.columns:
        test_price_feat[col] = pd.to_numeric(test_price_feat[col], errors='coerce')
        test_price_feat[f'{col}_log'] = np.log1p(test_price_feat[col])

# 类型转换
for col in test_price_feat.select_dtypes(include=['object']).columns:
    if col == 'Price':
        continue
    try:
        test_price_feat[col] = pd.to_numeric(test_price_feat[col], errors='ignore')
    except:
        pass

# 第四部分：分离特征和目标变量

# 从训练数据中删除目标变量'Price'列，得到特征矩阵X
X_price = train_price_feat.drop(columns=['Price'])
# 提取目标变量'Price'作为y
y_price = train_price_feat['Price']

# 第五部分：类别变量编码

# 当前步骤
print("\n编码类别变量")

# 保存测试集的原始ID（用于最终提交）
test_price_ids = test_price_feat['ID'].copy()

# 从特征中删除ID列（ID不应该用于建模）
X_price_no_id = X_price.drop(columns=['ID'], errors='ignore')
# 从测试集中删除Price和ID列
test_price_no_id = test_price_feat.drop(columns=['Price', 'ID'], errors='ignore')

#  训练集编码
# 创建训练集副本
X_price_encoded = X_price_no_id.copy()

# 获取所有类别型特征（对象类型）
categorical_cols_encode = X_price_encoded.select_dtypes(include=['object']).columns.tolist()

# 初始化低基数和高基数类别列表
low_cardinality_cols = []  # 类别数少于10的列
high_cardinality_cols = []  # 类别数大于等于10的列

# 遍历所有类别列，根据唯一值数量分类
for col in categorical_cols_encode:
    # 如果该列的唯一值数量少于10
    if X_price_encoded[col].nunique() < 10:
        # 加入低基数列表（使用one-hot编码）
        low_cardinality_cols.append(col)
    else:
        # 加入高基数列表（使用频率编码）
        high_cardinality_cols.append(col)

#  One-hot编码（低基数特征）将每个类别变成单独0/1列
# 如果存在低基数特征：类别少，拆成多列也不会太多，而且能保留每个类别的独特信息
if low_cardinality_cols:
    # 使用pd.get_dummies进行one-hot编码，drop_first=True避免多重共线性
    X_price_encoded = pd.get_dummies(X_price_encoded, columns=low_cardinality_cols, drop_first=True)

#  频率编码（高基数特征）：：将类别替换成出现频率
# 创建字典存储频率编码映射
freq_encoding_dict = {}
# 遍历所有高基数列
for col in high_cardinality_cols:
    # 计算每个类别的频率（归一化）
    freq_encoding = X_price_encoded[col].value_counts(normalize=True).to_dict()
    # 保存频率编码字典，供测试集使用
    freq_encoding_dict[col] = freq_encoding
    # 使用频率编码替换原列，创建新的频率特征列
    X_price_encoded[f'{col}_freq'] = X_price_encoded[col].map(freq_encoding)
    # 删除原始类别列
    X_price_encoded = X_price_encoded.drop(columns=[col])

#  测试集编码
# 创建测试集副本
test_price_encoded = test_price_no_id.copy()

# One-hot编码（使用与训练集相同的列）
if low_cardinality_cols:
    test_price_encoded = pd.get_dummies(test_price_encoded, columns=low_cardinality_cols, drop_first=True)

# 频率编码（使用训练集的频率映射）
for col in high_cardinality_cols:
    # 使用训练集的频率字典映射测试集，未出现的类别填充0
    test_price_encoded[f'{col}_freq'] = test_price_encoded[col].map(freq_encoding_dict[col]).fillna(0)
    # 删除原始列
    test_price_encoded = test_price_encoded.drop(columns=[col], errors='ignore')

#  对齐训练集和测试集的列
# 找出训练集有但测试集没有的列
missing_cols = set(X_price_encoded.columns) - set(test_price_encoded.columns)
# 为测试集添加缺失的列，并填充0
for col in missing_cols:
    test_price_encoded[col] = 0

# 找出测试集有但训练集没有的列
extra_cols = set(test_price_encoded.columns) - set(X_price_encoded.columns)
# 删除测试集中多余的列
test_price_encoded = test_price_encoded.drop(columns=list(extra_cols))

# 确保测试集的列顺序与训练集一致
test_price_encoded = test_price_encoded[X_price_encoded.columns]

# 编码后的特征数量
print(f"编码后特征数量: {X_price_encoded.shape[1]}")

# 第六部分：分割训练集和验证集

# 当前步骤
print("\n分割数据集（80% 训练，20% 测试）")
# 使用train_test_split将数据分为80%训练，20%验证
# random_state=111确保结果可重复
X_price_train, X_price_test, y_price_train, y_price_test = train_test_split(
    X_price_encoded, y_price, test_size=0.2, random_state=111
)

# 训练集大小
print(f"训练集大小: {X_price_train.shape}")
# 验证集大小
print(f"测试集大小: {X_price_test.shape}")

# 第七部分：异常值处理

# 当前步骤
print("\n异常值检测和处理")
# 记录原始训练集大小
original_train_size = len(X_price_train)

#  使用IQR方法检测异常值
# 计算目标变量的第一四分位数（25%分位数）
Q1 = y_price_train.quantile(0.25)
# 计算目标变量的第三四分位数（75%分位数）
Q3 = y_price_train.quantile(0.75)
# 计算四分位距 IQR = Q3 - Q1
IQR = Q3 - Q1
# 计算下界：Q1 - 1.5*IQR
lower_bound = Q1 - 1.5 * IQR
# 计算上界：Q3 + 1.5*IQR
upper_bound = Q3 + 1.5 * IQR

# 创建布尔掩码：标记不是异常值的数据点
# 保留目标变量在[下界, 上界]范围内的数据
mask = (y_price_train >= lower_bound) & (y_price_train <= upper_bound)
# 根据掩码过滤特征矩阵
X_price_train_clean = X_price_train[mask]
# 根据掩码过滤目标变量
y_price_train_clean = y_price_train[mask]

# 异常值处理后的训练集大小
print(f"异常值处理后训练集大小: {X_price_train_clean.shape}")
# 总的预测数量（去除异常值后的训练集 + 验证集）
print(f"总预测数量（去除异常值后）: {len(X_price_train_clean) + len(X_price_test)}")

# 第八部分：特征选择

# 当前步骤
print("\n特征选择")
# 计算每个特征与目标变量的相关系数（皮尔逊相关系数）
# corrwith计算DataFrame与Series之间的相关性
# abs()取绝对值，因为负相关也很重要
# sort_values降序排列
correlation_with_target = X_price_train_clean.corrwith(y_price_train_clean).abs().sort_values(ascending=False)

# 相关性最高的前10个特征
print("\n与目标变量相关性最高的前10个特征:")
print(correlation_with_target.head(10))

# 选择相关性大于0.01的特征
# 这个阈值可以根据实际情况调整
selected_features = correlation_with_target[correlation_with_target > 0.01].index.tolist()
# 选择的特征数量
print(f"\n选择相关性>0.01的特征，共 {len(selected_features)} 个")

# 根据选择的特征过滤训练集
X_price_train_selected = X_price_train_clean[selected_features]
# 根据选择的特征过滤验证集
X_price_test_selected = X_price_test[selected_features]
# 根据选择的特征过滤最终测试集
test_price_selected = test_price_encoded[selected_features]

# 第九部分：数据标准化

# 创建标准化器对象，用于将特征缩放到均值为0，标准差为1
scaler_price = StandardScaler()
# 在训练集上拟合标准化器（学习均值和标准差）并转换训练集
X_price_train_scaled = scaler_price.fit_transform(X_price_train_selected)
# 使用训练集的参数转换验证集
X_price_test_scaled = scaler_price.transform(X_price_test_selected)

# 第十部分：训练和评估模型

# 当前步骤
print("\n训练和评估多个模型")
# 设置交叉验证的折数
cv_folds = 6
# 创建字典存储模型结果
results_price = {}

#  1. 普通最小二乘法（OLS）线性回归
# 当前训练的模型
print("\n训练 OLS 模型")
# 创建线性回归模型对象
model_ols = LinearRegression()
# 在训练集上训练模型
model_ols.fit(X_price_train_scaled, y_price_train_clean)

# 在训练集上进行预测
y_train_pred_ols = model_ols.predict(X_price_train_scaled)
# 在验证集上进行预测
y_test_pred_ols = model_ols.predict(X_price_test_scaled)

# 计算训练集的平均绝对误差（MAE）
mae_train_ols = mean_absolute_error(y_price_train_clean, y_train_pred_ols)
# 计算验证集的平均绝对误差
mae_test_ols = mean_absolute_error(y_price_test, y_test_pred_ols)

# 使用交叉验证评估模型
# cv=cv_folds表示使用6折交叉验证
# scoring='neg_mean_absolute_error'使用负的MAE作为评分（sklearn中分数越高越好）
cv_scores_ols = cross_val_score(model_ols, X_price_train_scaled, y_price_train_clean,
                                cv=cv_folds, scoring='neg_mean_absolute_error')
# 取负号得到正的MAE，并计算平均值
mae_cv_ols = -cv_scores_ols.mean()

# OLS模型的评估结果
print(f"  训练集 MAE: {mae_train_ols:.2f}")
print(f"  测试集 MAE: {mae_test_ols:.2f}")
print(f"  交叉验证 MAE: {mae_cv_ols:.2f}")

# 将OLS模型的结果存储到字典中
results_price['OLS'] = {
    'model': model_ols,  # 模型对象
    'mae_train': mae_train_ols,  # 训练集MAE
    'mae_test': mae_test_ols,  # 测试集MAE
    'mae_cv': mae_cv_ols  # 交叉验证MAE
}

#  2. Lasso回归（L1正则化）
# 当前训练的模型
print("\n训练 Lasso 模型")
# 创建Lasso回归模型，alpha是正则化强度，max_iter是最大迭代次数
model_lasso = Lasso(alpha=1.0, max_iter=10000, random_state=111)
# 训练模型
model_lasso.fit(X_price_train_scaled, y_price_train_clean)

# 预测
y_train_pred_lasso = model_lasso.predict(X_price_train_scaled)
y_test_pred_lasso = model_lasso.predict(X_price_test_scaled)

# 计算MAE
mae_train_lasso = mean_absolute_error(y_price_train_clean, y_train_pred_lasso)
mae_test_lasso = mean_absolute_error(y_price_test, y_test_pred_lasso)

# 交叉验证
cv_scores_lasso = cross_val_score(model_lasso, X_price_train_scaled, y_price_train_clean,
                                  cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_lasso = -cv_scores_lasso.mean()

# 结果
print(f"  训练集 MAE: {mae_train_lasso:.2f}")
print(f"  测试集 MAE: {mae_test_lasso:.2f}")
print(f"  交叉验证 MAE: {mae_cv_lasso:.2f}")

# 存储结果
results_price['Lasso'] = {
    'model': model_lasso,
    'mae_train': mae_train_lasso,
    'mae_test': mae_test_lasso,
    'mae_cv': mae_cv_lasso
}

#  3. Ridge回归（L2正则化）
# 当前训练的模型
print("\n训练 Ridge 模型")
# 创建Ridge回归模型
model_ridge = Ridge(alpha=1.0, max_iter=10000, random_state=111)
# 训练模型
model_ridge.fit(X_price_train_scaled, y_price_train_clean)

# 预测
y_train_pred_ridge = model_ridge.predict(X_price_train_scaled)
y_test_pred_ridge = model_ridge.predict(X_price_test_scaled)

# 计算MAE
mae_train_ridge = mean_absolute_error(y_price_train_clean, y_train_pred_ridge)
mae_test_ridge = mean_absolute_error(y_price_test, y_test_pred_ridge)

# 交叉验证
cv_scores_ridge = cross_val_score(model_ridge, X_price_train_scaled, y_price_train_clean,
                                  cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_ridge = -cv_scores_ridge.mean()

# 结果
print(f"训练集 MAE: {mae_train_ridge:.2f}")
print(f"测试集 MAE: {mae_test_ridge:.2f}")
print(f"交叉验证 MAE: {mae_cv_ridge:.2f}")

# 存储结果
results_price['Ridge'] = {
    'model': model_ridge,
    'mae_train': mae_train_ridge,
    'mae_test': mae_test_ridge,
    'mae_cv': mae_cv_ridge
}

#  4. ElasticNet回归（L1+L2正则化）
# 当前训练的模型
print("\n训练 ElasticNet 模型")
# 创建ElasticNet模型，l1_ratio控制L1和L2的比例
model_elasticnet = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000, random_state=111)
# 训练模型
model_elasticnet.fit(X_price_train_scaled, y_price_train_clean)

# 预测
y_train_pred_en = model_elasticnet.predict(X_price_train_scaled)
y_test_pred_en = model_elasticnet.predict(X_price_test_scaled)

# 计算MAE
mae_train_en = mean_absolute_error(y_price_train_clean, y_train_pred_en)
mae_test_en = mean_absolute_error(y_price_test, y_test_pred_en)

# 交叉验证
cv_scores_en = cross_val_score(model_elasticnet, X_price_train_scaled, y_price_train_clean,
                               cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_en = -cv_scores_en.mean()

# 结果
print(f"训练集 MAE: {mae_train_en:.2f}")
print(f"测试集 MAE: {mae_test_en:.2f}")
print(f"交叉验证 MAE: {mae_cv_en:.2f}")

# 存储结果
results_price['ElasticNet'] = {
    'model': model_elasticnet,
    'mae_train': mae_train_en,
    'mae_test': mae_test_en,
    'mae_cv': mae_cv_en
}

# 保存标准化器，供后续使用
results_price['scaler'] = scaler_price

# 第十一部分：模型比较结果表格
print("模型性能比较")
print(f"{'模型':<15} {'训练集MAE':<15} {'测试集MAE':<15} {'交叉验证MAE':<15}")
# 分隔线
print("-" * 80)

# 遍历所有模型，结果
for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    # 获取该模型的结果
    r = results_price[name]
    print(f"{name:<15} {r['mae_train']:<15.2f} {r['mae_test']:<15.2f} {r['mae_cv']:<15.2f}")

#  选择最佳模型
# 初始化最佳模型名称
best_model_name_price = None
# 初始化最佳测试集MAE为无穷大
best_mae_price = float('inf')

# 遍历所有模型，找出测试集MAE最小的模型
for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    # 如果该模型的测试集MAE小于当前最佳MAE
    if results_price[name]['mae_test'] < best_mae_price:
        # 更新最佳MAE
        best_mae_price = results_price[name]['mae_test']
        # 更新最佳模型名称
        best_model_name_price = name

# 最佳模型信息
print(f"最佳模型（基于测试集MAE）: {best_model_name_price}")

# 第十二部分：处理租金数据 - 数据清洗

print("处理租金数据")

# 当前步骤
print("\n数据清洗")
# 创建训练集副本
train_rent_clean = train_rent.copy()

#  识别并删除数据泄露特征（租金）
# 初始化泄露特征列表
leakage_features_rent = []
# 关键词列表（与房价相同）
keywords_rent = ['comm', 'price', '价格', '反馈', '客户']

# 遍历所有列
for col in train_rent_clean.columns:
    if col == 'Price':
        continue
    col_lower = col.lower()
    # 检查是否包含关键词
    for keyword in keywords_rent:
        if keyword in col_lower:
            leakage_features_rent.append(col)
            break

# 识别到的泄露特征
print(f"  识别到可能的数据泄露特征: {leakage_features_rent}")
# 删除泄露特征
train_rent_clean = train_rent_clean.drop(columns=leakage_features_rent, errors='ignore')

#  处理缺失值（租金训练集）
# 数值型列用中位数填充
numeric_cols_rent = train_rent_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols_rent:
    if train_rent_clean[col].isnull().sum() > 0:
        train_rent_clean[col].fillna(train_rent_clean[col].median(), inplace=True)

# 类别型列用众数填充
categorical_cols_rent = train_rent_clean.select_dtypes(include=['object']).columns
for col in categorical_cols_rent:
    if train_rent_clean[col].isnull().sum() > 0:
        train_rent_clean[col].fillna(train_rent_clean[col].mode()[0], inplace=True)

# 清洗后的数据维度
print(f"清洗后训练数据: {train_rent_clean.shape}")

#  清洗测试集（租金）
# 创建测试集副本
test_rent_clean = test_rent.copy()
# 删除泄露特征
test_rent_clean = test_rent_clean.drop(columns=leakage_features_rent, errors='ignore')

# 处理缺失值
numeric_cols_rent_test = test_rent_clean.select_dtypes(include=[np.number]).columns
for col in numeric_cols_rent_test:
    if test_rent_clean[col].isnull().sum() > 0:
        test_rent_clean[col].fillna(test_rent_clean[col].median(), inplace=True)

categorical_cols_rent_test = test_rent_clean.select_dtypes(include=['object']).columns
for col in categorical_cols_rent_test:
    if test_rent_clean[col].isnull().sum() > 0:
        test_rent_clean[col].fillna(test_rent_clean[col].mode()[0], inplace=True)

# 第十三部分：租金数据的特征工程

# 当前步骤
print("\n特征工程")
# 创建副本
train_rent_feat = train_rent_clean.copy()

#  提取房屋户型信息
if '房屋户型' in train_rent_feat.columns:
    train_rent_feat['室数'] = train_rent_feat['房屋户型'].str.extract(r'(\d+)室').astype(float)
    train_rent_feat['厅数'] = train_rent_feat['房屋户型'].str.extract(r'(\d+)厅').astype(float)
    train_rent_feat['卫数'] = train_rent_feat['房屋户型'].str.extract(r'(\d+)卫').astype(float)
    train_rent_feat['室数'].fillna(0, inplace=True)
    train_rent_feat['厅数'].fillna(0, inplace=True)
    train_rent_feat['卫数'].fillna(0, inplace=True)

#  提取楼层信息
if '所在楼层' in train_rent_feat.columns:
    train_rent_feat['是否高层'] = train_rent_feat['所在楼层'].str.contains('高', na=False).astype(int)
    train_rent_feat['是否低层'] = train_rent_feat['所在楼层'].str.contains('低', na=False).astype(int)
    train_rent_feat['是否中层'] = train_rent_feat['所在楼层'].str.contains('中', na=False).astype(int)

#  提取梯户比例
if '梯户比例' in train_rent_feat.columns:
    train_rent_feat['梯数'] = train_rent_feat['梯户比例'].str.extract(r'(\d+)梯').astype(float)
    train_rent_feat['户数'] = train_rent_feat['梯户比例'].str.extract(r'(\d+)户').astype(float)
    train_rent_feat['梯数'].fillna(train_rent_feat['梯数'].median(), inplace=True)
    train_rent_feat['户数'].fillna(train_rent_feat['户数'].median(), inplace=True)
    train_rent_feat['梯户比'] = train_rent_feat['梯数'] / (train_rent_feat['户数'] + 1)

#  创建面积特征
if '建筑面积' in train_rent_feat.columns and '套内面积' in train_rent_feat.columns:
    train_rent_feat['建筑面积'] = pd.to_numeric(train_rent_feat['建筑面积'], errors='coerce')
    train_rent_feat['套内面积'] = pd.to_numeric(train_rent_feat['套内面积'], errors='coerce')
    train_rent_feat['公摊面积'] = train_rent_feat['建筑面积'] - train_rent_feat['套内面积']
    train_rent_feat['公摊率'] = train_rent_feat['公摊面积'] / (train_rent_feat['建筑面积'] + 1)

#  创建时间特征
if '年份' in train_rent_feat.columns:
    train_rent_feat['年份'] = pd.to_numeric(train_rent_feat['年份'], errors='coerce')
    train_rent_feat['房龄'] = 2025 - train_rent_feat['年份']

if '建筑年代' in train_rent_feat.columns:
    train_rent_feat['建筑年份'] = pd.to_numeric(train_rent_feat['建筑年代'], errors='coerce')
    train_rent_feat['建筑年份'].fillna(train_rent_feat['建筑年份'].median(), inplace=True)
    train_rent_feat['建筑物房龄'] = 2025 - train_rent_feat['建筑年份']

#  创建交互特征
if '建筑面积' in train_rent_feat.columns and '室数' in train_rent_feat.columns:
    train_rent_feat['平均房间面积'] = train_rent_feat['建筑面积'] / (train_rent_feat['室数'] + 1)

#  创建位置特征
if 'lon' in train_rent_feat.columns and 'lat' in train_rent_feat.columns:
    train_rent_feat['lon'] = pd.to_numeric(train_rent_feat['lon'], errors='coerce')
    train_rent_feat['lat'] = pd.to_numeric(train_rent_feat['lat'], errors='coerce')
    center_lon_rent = train_rent_feat['lon'].median()
    center_lat_rent = train_rent_feat['lat'].median()
    train_rent_feat['距市中心距离'] = np.sqrt(
        (train_rent_feat['lon'] - center_lon_rent) ** 2 + (train_rent_feat['lat'] - center_lat_rent) ** 2
    )

#  对数转换
numeric_cols_for_log_rent = ['建筑面积', '套内面积', '房屋总数', '楼栋总数']
for col in numeric_cols_for_log_rent:
    if col in train_rent_feat.columns:
        train_rent_feat[col] = pd.to_numeric(train_rent_feat[col], errors='coerce')
        train_rent_feat[f'{col}_log'] = np.log1p(train_rent_feat[col])

#  类型转换
for col in train_rent_feat.select_dtypes(include=['object']).columns:
    if col == 'Price':
        continue
    try:
        train_rent_feat[col] = pd.to_numeric(train_rent_feat[col], errors='ignore')
    except:
        pass

# 特征工程后的维度
print(f"特征工程后训练数据: {train_rent_feat.shape}")

#  对测试集应用相同的特征工程
test_rent_feat = test_rent_clean.copy()

# 提取房屋户型
if '房屋户型' in test_rent_feat.columns:
    test_rent_feat['室数'] = test_rent_feat['房屋户型'].str.extract(r'(\d+)室').astype(float)
    test_rent_feat['厅数'] = test_rent_feat['房屋户型'].str.extract(r'(\d+)厅').astype(float)
    test_rent_feat['卫数'] = test_rent_feat['房屋户型'].str.extract(r'(\d+)卫').astype(float)
    test_rent_feat['室数'].fillna(0, inplace=True)
    test_rent_feat['厅数'].fillna(0, inplace=True)
    test_rent_feat['卫数'].fillna(0, inplace=True)

# 提取楼层信息
if '所在楼层' in test_rent_feat.columns:
    test_rent_feat['是否高层'] = test_rent_feat['所在楼层'].str.contains('高', na=False).astype(int)
    test_rent_feat['是否低层'] = test_rent_feat['所在楼层'].str.contains('低', na=False).astype(int)
    test_rent_feat['是否中层'] = test_rent_feat['所在楼层'].str.contains('中', na=False).astype(int)

# 提取梯户比例
if '梯户比例' in test_rent_feat.columns:
    test_rent_feat['梯数'] = test_rent_feat['梯户比例'].str.extract(r'(\d+)梯').astype(float)
    test_rent_feat['户数'] = test_rent_feat['梯户比例'].str.extract(r'(\d+)户').astype(float)
    test_rent_feat['梯数'].fillna(test_rent_feat['梯数'].median(), inplace=True)
    test_rent_feat['户数'].fillna(test_rent_feat['户数'].median(), inplace=True)
    test_rent_feat['梯户比'] = test_rent_feat['梯数'] / (test_rent_feat['户数'] + 1)

# 创建面积特征
if '建筑面积' in test_rent_feat.columns and '套内面积' in test_rent_feat.columns:
    test_rent_feat['建筑面积'] = pd.to_numeric(test_rent_feat['建筑面积'], errors='coerce')
    test_rent_feat['套内面积'] = pd.to_numeric(test_rent_feat['套内面积'], errors='coerce')
    test_rent_feat['公摊面积'] = test_rent_feat['建筑面积'] - test_rent_feat['套内面积']
    test_rent_feat['公摊率'] = test_rent_feat['公摊面积'] / (test_rent_feat['建筑面积'] + 1)

# 创建时间特征
if '年份' in test_rent_feat.columns:
    test_rent_feat['年份'] = pd.to_numeric(test_rent_feat['年份'], errors='coerce')
    test_rent_feat['房龄'] = 2025 - test_rent_feat['年份']

if '建筑年代' in test_rent_feat.columns:
    test_rent_feat['建筑年份'] = pd.to_numeric(test_rent_feat['建筑年代'], errors='coerce')
    test_rent_feat['建筑年份'].fillna(test_rent_feat['建筑年份'].median(), inplace=True)
    test_rent_feat['建筑物房龄'] = 2025 - test_rent_feat['建筑年份']

# 创建交互特征
if '建筑面积' in test_rent_feat.columns and '室数' in test_rent_feat.columns:
    test_rent_feat['平均房间面积'] = test_rent_feat['建筑面积'] / (test_rent_feat['室数'] + 1)

# 创建位置特征（使用训练集的中心点）
if 'lon' in test_rent_feat.columns and 'lat' in test_rent_feat.columns:
    test_rent_feat['lon'] = pd.to_numeric(test_rent_feat['lon'], errors='coerce')
    test_rent_feat['lat'] = pd.to_numeric(test_rent_feat['lat'], errors='coerce')
    test_rent_feat['距市中心距离'] = np.sqrt(
        (test_rent_feat['lon'] - center_lon_rent) ** 2 + (test_rent_feat['lat'] - center_lat_rent) ** 2
    )

# 对数转换
for col in numeric_cols_for_log_rent:
    if col in test_rent_feat.columns:
        test_rent_feat[col] = pd.to_numeric(test_rent_feat[col], errors='coerce')
        test_rent_feat[f'{col}_log'] = np.log1p(test_rent_feat[col])

# 类型转换
for col in test_rent_feat.select_dtypes(include=['object']).columns:
    if col == 'Price':
        continue
    try:
        test_rent_feat[col] = pd.to_numeric(test_rent_feat[col], errors='ignore')
    except:
        pass

# 第十四部分：租金数据 - 分离特征和目标变量

# 分离特征和目标
X_rent = train_rent_feat.drop(columns=['Price'])
y_rent = train_rent_feat['Price']

# 第十五部分：租金数据 - 类别变量编码

# 当前步骤
print("\n编码类别变量")

# 保存测试集的原始ID（用于最终提交）
test_rent_ids = test_rent_feat['ID'].copy()

# 删除ID列
X_rent_no_id = X_rent.drop(columns=['ID'], errors='ignore')
test_rent_no_id = test_rent_feat.drop(columns=['Price', 'ID'], errors='ignore')

#  训练集编码
X_rent_encoded = X_rent_no_id.copy()

# 获取类别型列
categorical_cols_encode_rent = X_rent_encoded.select_dtypes(include=['object']).columns.tolist()

# 分类为低基数和高基数
low_cardinality_cols_rent = []
high_cardinality_cols_rent = []

for col in categorical_cols_encode_rent:
    if X_rent_encoded[col].nunique() < 10:
        low_cardinality_cols_rent.append(col)
    else:
        high_cardinality_cols_rent.append(col)

# One-hot编码
if low_cardinality_cols_rent:
    X_rent_encoded = pd.get_dummies(X_rent_encoded, columns=low_cardinality_cols_rent, drop_first=True)

# 频率编码
freq_encoding_dict_rent = {}
for col in high_cardinality_cols_rent:
    freq_encoding = X_rent_encoded[col].value_counts(normalize=True).to_dict()
    freq_encoding_dict_rent[col] = freq_encoding
    X_rent_encoded[f'{col}_freq'] = X_rent_encoded[col].map(freq_encoding)
    X_rent_encoded = X_rent_encoded.drop(columns=[col])

#  测试集编码
test_rent_encoded = test_rent_no_id.copy()

# One-hot编码
if low_cardinality_cols_rent:
    test_rent_encoded = pd.get_dummies(test_rent_encoded, columns=low_cardinality_cols_rent, drop_first=True)

# 频率编码
for col in high_cardinality_cols_rent:
    test_rent_encoded[f'{col}_freq'] = test_rent_encoded[col].map(freq_encoding_dict_rent[col]).fillna(0)
    test_rent_encoded = test_rent_encoded.drop(columns=[col], errors='ignore')

#  对齐列
missing_cols_rent = set(X_rent_encoded.columns) - set(test_rent_encoded.columns)
for col in missing_cols_rent:
    test_rent_encoded[col] = 0

extra_cols_rent = set(test_rent_encoded.columns) - set(X_rent_encoded.columns)
test_rent_encoded = test_rent_encoded.drop(columns=list(extra_cols_rent))

test_rent_encoded = test_rent_encoded[X_rent_encoded.columns]

# 编码后特征数量
print(f"编码后特征数量: {X_rent_encoded.shape[1]}")

# 第十六部分：租金数据 - 分割训练集和验证集

# 当前步骤
print("\n分割数据集（80% 训练，20% 测试）")
X_rent_train, X_rent_test, y_rent_train, y_rent_test = train_test_split(
    X_rent_encoded, y_rent, test_size=0.2, random_state=111
)

# 分割后的大小
print(f"训练集大小: {X_rent_train.shape}")
print(f"测试集大小: {X_rent_test.shape}")

# 第十七部分：租金数据 - 异常值处理

# 当前步骤
print("\n异常值检测和处理")

# 使用IQR方法
Q1_rent = y_rent_train.quantile(0.25)
Q3_rent = y_rent_train.quantile(0.75)
IQR_rent = Q3_rent - Q1_rent
lower_bound_rent = Q1_rent - 1.5 * IQR_rent
upper_bound_rent = Q3_rent + 1.5 * IQR_rent

# 创建掩码并过滤
mask_rent = (y_rent_train >= lower_bound_rent) & (y_rent_train <= upper_bound_rent)
X_rent_train_clean = X_rent_train[mask_rent]
y_rent_train_clean = y_rent_train[mask_rent]

# 处理后的大小
print(f"异常值处理后训练集大小: {X_rent_train_clean.shape}")
print(f"总预测数量（去除异常值后）: {len(X_rent_train_clean) + len(X_rent_test)}")

# 第十八部分：租金数据 - 特征选择

# 当前步骤
print("\n特征选择")
# 计算相关性
correlation_with_target_rent = X_rent_train_clean.corrwith(y_rent_train_clean).abs().sort_values(ascending=False)

# 前10个最相关的特征
print("\n与目标变量相关性最高的前10个特征:")
print(correlation_with_target_rent.head(10))

# 选择相关性大于0.01的特征
selected_features_rent = correlation_with_target_rent[correlation_with_target_rent > 0.01].index.tolist()
print(f"\n选择相关性>0.01的特征，共 {len(selected_features_rent)} 个")

# 根据选择的特征过滤数据
X_rent_train_selected = X_rent_train_clean[selected_features_rent]
X_rent_test_selected = X_rent_test[selected_features_rent]
test_rent_selected = test_rent_encoded[selected_features_rent]

# 第十九部分：租金数据 - 标准化

# 创建标准化器
scaler_rent = StandardScaler()
# 拟合并转换训练集
X_rent_train_scaled = scaler_rent.fit_transform(X_rent_train_selected)
# 转换验证集
X_rent_test_scaled = scaler_rent.transform(X_rent_test_selected)

# 第二十部分：租金数据 - 训练和评估模型

# 当前步骤
print("\n训练和评估多个模型")
# 创建结果字典
results_rent = {}

#  1. OLS线性回归
print("\n训练 OLS 模型")
model_ols_rent = LinearRegression()
model_ols_rent.fit(X_rent_train_scaled, y_rent_train_clean)

y_train_pred_ols_rent = model_ols_rent.predict(X_rent_train_scaled)
y_test_pred_ols_rent = model_ols_rent.predict(X_rent_test_scaled)

mae_train_ols_rent = mean_absolute_error(y_rent_train_clean, y_train_pred_ols_rent)
mae_test_ols_rent = mean_absolute_error(y_rent_test, y_test_pred_ols_rent)

cv_scores_ols_rent = cross_val_score(model_ols_rent, X_rent_train_scaled, y_rent_train_clean,
                                     cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_ols_rent = -cv_scores_ols_rent.mean()

print(f"  训练集 MAE: {mae_train_ols_rent:.2f}")
print(f"  测试集 MAE: {mae_test_ols_rent:.2f}")
print(f"  交叉验证 MAE: {mae_cv_ols_rent:.2f}")

results_rent['OLS'] = {
    'model': model_ols_rent,
    'mae_train': mae_train_ols_rent,
    'mae_test': mae_test_ols_rent,
    'mae_cv': mae_cv_ols_rent
}

#  2. Lasso回归
print("\n训练 Lasso 模型")
model_lasso_rent = Lasso(alpha=1.0, max_iter=10000, random_state=111)
model_lasso_rent.fit(X_rent_train_scaled, y_rent_train_clean)

y_train_pred_lasso_rent = model_lasso_rent.predict(X_rent_train_scaled)
y_test_pred_lasso_rent = model_lasso_rent.predict(X_rent_test_scaled)

mae_train_lasso_rent = mean_absolute_error(y_rent_train_clean, y_train_pred_lasso_rent)
mae_test_lasso_rent = mean_absolute_error(y_rent_test, y_test_pred_lasso_rent)

cv_scores_lasso_rent = cross_val_score(model_lasso_rent, X_rent_train_scaled, y_rent_train_clean,
                                       cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_lasso_rent = -cv_scores_lasso_rent.mean()

print(f"  训练集 MAE: {mae_train_lasso_rent:.2f}")
print(f"  测试集 MAE: {mae_test_lasso_rent:.2f}")
print(f"  交叉验证 MAE: {mae_cv_lasso_rent:.2f}")

results_rent['Lasso'] = {
    'model': model_lasso_rent,
    'mae_train': mae_train_lasso_rent,
    'mae_test': mae_test_lasso_rent,
    'mae_cv': mae_cv_lasso_rent
}

#  3. Ridge回归
print("\n训练 Ridge 模型")
model_ridge_rent = Ridge(alpha=1.0, max_iter=10000, random_state=111)
model_ridge_rent.fit(X_rent_train_scaled, y_rent_train_clean)

y_train_pred_ridge_rent = model_ridge_rent.predict(X_rent_train_scaled)
y_test_pred_ridge_rent = model_ridge_rent.predict(X_rent_test_scaled)

mae_train_ridge_rent = mean_absolute_error(y_rent_train_clean, y_train_pred_ridge_rent)
mae_test_ridge_rent = mean_absolute_error(y_rent_test, y_test_pred_ridge_rent)

cv_scores_ridge_rent = cross_val_score(model_ridge_rent, X_rent_train_scaled, y_rent_train_clean,
                                       cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_ridge_rent = -cv_scores_ridge_rent.mean()

print(f"  训练集 MAE: {mae_train_ridge_rent:.2f}")
print(f"  测试集 MAE: {mae_test_ridge_rent:.2f}")
print(f"  交叉验证 MAE: {mae_cv_ridge_rent:.2f}")

results_rent['Ridge'] = {
    'model': model_ridge_rent,
    'mae_train': mae_train_ridge_rent,
    'mae_test': mae_test_ridge_rent,
    'mae_cv': mae_cv_ridge_rent
}

#  4. ElasticNet回归
print("\n训练 ElasticNet 模型")
model_elasticnet_rent = ElasticNet(alpha=1.0, l1_ratio=0.5, max_iter=10000, random_state=111)
model_elasticnet_rent.fit(X_rent_train_scaled, y_rent_train_clean)

y_train_pred_en_rent = model_elasticnet_rent.predict(X_rent_train_scaled)
y_test_pred_en_rent = model_elasticnet_rent.predict(X_rent_test_scaled)

mae_train_en_rent = mean_absolute_error(y_rent_train_clean, y_train_pred_en_rent)
mae_test_en_rent = mean_absolute_error(y_rent_test, y_test_pred_en_rent)

cv_scores_en_rent = cross_val_score(model_elasticnet_rent, X_rent_train_scaled, y_rent_train_clean,
                                    cv=cv_folds, scoring='neg_mean_absolute_error')
mae_cv_en_rent = -cv_scores_en_rent.mean()

print(f"  训练集 MAE: {mae_train_en_rent:.2f}")
print(f"  测试集 MAE: {mae_test_en_rent:.2f}")
print(f"  交叉验证 MAE: {mae_cv_en_rent:.2f}")

results_rent['ElasticNet'] = {
    'model': model_elasticnet_rent,
    'mae_train': mae_train_en_rent,
    'mae_test': mae_test_en_rent,
    'mae_cv': mae_cv_en_rent
}

# 保存标准化器
results_rent['scaler'] = scaler_rent

# 第二十一部分：租金数据 - 模型比较结果

# 分隔线
print("\n" + "=" * 80)
print("模型性能比较")
print("=" * 80)
# 表头
print(f"{'模型':<15} {'训练集MAE':<15} {'测试集MAE':<15} {'交叉验证MAE':<15}")
print("-" * 80)

# 遍历所有模型结果
for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    r = results_rent[name]
    print(f"{name:<15} {r['mae_train']:<15.2f} {r['mae_test']:<15.2f} {r['mae_cv']:<15.2f}")

#  选择最佳模型
best_model_name_rent = None
best_mae_rent = float('inf')

for name in ['OLS', 'Lasso', 'Ridge', 'ElasticNet']:
    if results_rent[name]['mae_test'] < best_mae_rent:
        best_mae_rent = results_rent[name]['mae_test']
        best_model_name_rent = name

# 最佳模型
print("=" * 80)
print(f"最佳模型（基于测试集MAE）: {best_model_name_rent}")
print("=" * 80)

# 第二十二部分：最终预测结果

# 获取标准化器
scaler_price_final = results_price['scaler']
scaler_rent_final = results_rent['scaler']

# 对测试集进行标准化
test_price_scaled_final = scaler_price_final.transform(test_price_selected)
test_rent_scaled_final = scaler_rent_final.transform(test_rent_selected)

# 使用之前保存的原始测试集ID（保证顺序与预测结果一致）
price_ids = test_price_ids.values
rent_ids = test_rent_ids.values

# 合并所有ID（房价ID + 租金ID）
all_ids = list(price_ids) + list(rent_ids)

# 遍历4种模型，为每种模型生成单独的预测文件
model_names = ['OLS', 'Lasso', 'Ridge', 'ElasticNet']

for model_name in model_names:
    #  房价预测
    # 当前使用的模型
    print(f"\n使用{model_name}模型进行价格预测")
    # 获取模型对象
    price_model = results_price[model_name]['model']
    # 使用模型进行预测
    price_predictions = price_model.predict(test_price_scaled_final)

    #  租金预测
    # 当前使用的模型
    print(f"使用{model_name}模型进行租金预测")
    # 获取模型对象
    rent_model = results_rent[model_name]['model']
    # 使用模型进行预测
    rent_predictions = rent_model.predict(test_rent_scaled_final)

    #  创建提交文件
    # 创建空的DataFrame
    submission = pd.DataFrame()

    # 合并所有预测值（房价预测 + 租金预测）
    all_predictions = list(price_predictions) + list(rent_predictions)

    # 将ID和预测值添加到提交DataFrame中
    submission['ID'] = all_ids
    submission['Price'] = all_predictions

    # 确保ID列是整数类型
    submission['ID'] = submission['ID'].astype(int)

    # 保存提交文件到outputs目录
    submission.to_csv(f'submission_{model_name}.csv', index=False)

    print(f"预测结果已保存到 submission_{model_name}.csv")




第一步：加载数据
训练数据(价格): (103871, 55)
测试数据(价格): (34017, 55)
训练数据(租金): (98899, 46)
测试数据(租金): (9773, 46)
处理房价数据

数据清洗
  识别到可能的数据泄露特征: ['板块_comm', '建筑结构_comm', '客户反馈']
清洗后训练数据: (103871, 52)

特征工程
特征工程后训练数据: (103871, 72)

编码类别变量
编码后特征数量: 87

分割数据集（80% 训练，20% 测试）
训练集大小: (83096, 87)
测试集大小: (20775, 87)

异常值检测和处理
异常值处理后训练集大小: (76784, 87)
总预测数量（去除异常值后）: 97559

特征选择

与目标变量相关性最高的前10个特征:
环线位置_freq      0.378053
环线_freq        0.366500
lon            0.337325
coord_x        0.325315
上次交易_freq      0.274456
燃气费_freq       0.258004
产权描述_freq      0.251099
物业办公电话_freq    0.230016
卫数             0.221258
室数             0.206183
dtype: float64

选择相关性>0.01的特征，共 64 个

训练和评估多个模型

训练 OLS 模型
  训练集 MAE: 586154.49
  测试集 MAE: 951123.82
  交叉验证 MAE: 586722.58

训练 Lasso 模型
  训练集 MAE: 586152.27
  测试集 MAE: 951121.98
  交叉验证 MAE: 586736.98

训练 Ridge 模型
训练集 MAE: 586152.64
测试集 MAE: 951122.81
交叉验证 MAE: 586737.25

训练 ElasticNet 模型
训练集 MAE: 619344.11
测试集 MAE: 1013006.37
交叉验证 MAE: 619648.17
模型性能比较
模型              训练集MAE         